In [127]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from scipy.signal import *
from scipy import signal
import pandas as pd

In [128]:
SIGNAL_PATH = 'C:/Users/yash2/OneDrive/Desktop/IIIT Class/6th Semester/BSPA/Project/Main-Project/Dataset/'

In [129]:
#Patient Info for PPG Signals.

subject_dia = pd.read_csv('diabetesPatientInfo.csv')
subject_nor = pd.read_csv('normalPatientInfo.csv')

In [130]:
subject_dia['Label'] = 1
subject_nor['Label'] = 0

In [131]:
subject = pd.concat([subject_dia, subject_nor], axis=0)

In [132]:
m, n = subject.shape

In [133]:
persons = subject['subject_ID']
subject

,subject_ID,Sex(M/F),Age(year),Height(cm),Weight(kg),Heart Rate(b/m),BMI(kg/m^2),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),Hypertension,Diabetes,1,2,3,Selected,Label
0,31,0,66,150,57,81,25.33,182,102,3,1,0.68,0.81,0.70,2,1
1,217,0,52,158,59,58,23.63,127,83,1,2,0.69,1.26,0.80,2,1
2,218,0,63,153,60,79,25.63,149,78,2,2,0.82,0.99,0.89,2,1
3,219,1,59,165,68,78,24.98,149,92,2,2,1.02,1.05,0.84,2,1
4,220,0,55,155,50,96,20.81,174,104,3,2,0.63,0.65,0.66,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,414,0,24,168,55,87,19.49,109,68,0,0,0.93,1.34,1.40,3,0
55,415,1,24,180,70,77,21.60,111,70,0,0,1.15,1.38,1.19,2,0
56,416,0,25,156,47,79,19.31,93,57,0,0,0.96,0.94,1.01,3,0
57,418,1,25,173,63,67,21.05,106,69,0,0,0.96,0.87,1.06,3,0


In [134]:
print("Total number of patients info:", m)

Total number of patients info: 97


In [135]:
#Butter-Worth Filter

def butterFil(signal):

    filt = butter(5, [0.15, 12], 'bp', fs = 40, output = 'sos')
    final = []

    for i in range(0, len(signal), 18):
        filtered = sosfilt(filt, signal[i:i+18])
        for j in filtered:
            final.append(j)

    #Normalizing the output
    ppgMax = max(final)
    ppgMin = min(final)
    final = (final - ppgMin) / (ppgMax - ppgMin)
            
    t = np.linspace(0,60,len(final))
    
    ret = [final, t]
    return (ret)

In [136]:
#Derivative Filter

def derivativeFil(signal):

    sig = []
    der1 = []
    der2 = []
    
    #1st Derivative
    delta = 0.001
    for i in range(1, len(signal)-1):
        d = (signal[i+1]-signal[i-1])/(2*delta)
        der1.append(d)
    
    #2nd Derivative
    for i in range(1, len(der1)-1):
        d = (der1[i+1]-der1[i-1])/(delta**2)
        der2.append(d)
    
    return [der1, der2]

In [137]:
#Systolic Amplitude

def systolic_peak(signal):
    
    sys_peak = []
    for i in range(0, len(signal)-19, 19):
        sys_peak.append(max(signal[i:i+19]))
        
    return sum(sys_peak)/len(sys_peak)

In [138]:
#Diastolic Amplitude

def diastolic_peak(signal):
    dia_peak = []
    flag = 0
    count = 0
    for i in range(0, len(signal)-18, 18):
        for j in range(18):
            if signal[i+j+1]-signal[i+j]<0:
                flag = 1
            
            
            if signal[i+j+1]-signal[i+j]>0 and flag == 1:
                dia_peak.append(signal[i+j+1])
                flag = 0
                
    diaspeak = 0
    c = 0
    for i in range(0, len(dia_peak), 2):
        diaspeak += dia_peak[i]
        c += 1
        
    return diaspeak/c

In [139]:
def pulse_interval(signal, time):
    times = []
    for i in range(0, len(signal)-19, 19):
        times.append(time[signal.index(max(signal[i:i+19]))])
    
    intervals = []
    for i in range(len(times)-1):
        intervals.append(times[i+1]-times[i])
    return sum(intervals)/len(intervals)

In [140]:
feature_vector = []
for patient in range(m):

    # Load the Signal0
    subject_ID = str(subject.iloc[patient, 0])
    selected = str(subject.iloc[patient, 14])
    tempSubject = SIGNAL_PATH + subject_ID + '_' + selected + '.txt'
    raw_PPG = np.loadtxt(tempSubject)

    # Preprocessing - 4th Order Butter Filter
    new_PPGsignal = butterFil(raw_PPG)
    PPGsignal = new_PPGsignal[0]
    
    #Plotting the processed signal
    # plt.subplot(4, 1, 1)
    # plt.plot(PPGsignal[:100])
    # plt.grid()
    # plt.title(subject_ID)
    # plt.show()

    #Finding the 1st Derivative and the 2nd Derivative
    derFil = derivativeFil(PPGsignal)
    
    #Plotting the first derivative
    # plt.subplot(4, 1, 2)
    # plt.plot(derFil[0][:100])
    # plt.grid()
    # plt.title("First Derivative: " + subject_ID)
    # plt.show()

    #Plotting tghe second derivative
    # plt.subplot(4, 1, 3)
    # plt.plot(derFil[1][:100])
    # plt.grid()
    # plt.title("Second Derivative: " + subject_ID)
    # plt.show()  
     
    #Finding the A, B, C, D, E values
    a = []
    b = []
    c = []
    d = []
    e = []

    for i in range(0, len(derFil[1])-18, 18):

        a.append(derFil[1][i+3])
        b.append(derFil[1][i+5])
        c.append(derFil[1][i+7])
        d.append(derFil[1][i+9])
        e.append(derFil[1][i+11])
    
    av = sum(a)/len(a)
    bv = sum(b)/len(b)
    cv = sum(c)/len(c)
    dv = sum(d)/len(d)
    ev = sum(e)/len(e)

    #Finding Systolic amplitude
    SysPeak = systolic_peak(PPGsignal)
    #Finding DiaStolic Amplitude
    DiaPeak = diastolic_peak(PPGsignal)
    #Finding the Pulse Interval
    t = np.linspace(0, 2.1, len(raw_PPG))
    pi = pulse_interval(PPGsignal.tolist(), t.tolist())
    #Finding the AI
    AI = DiaPeak/SysPeak
    #Finding the Adjusted AI
    adj_AI = 1 - AI
    #Finding the ATI/SP
    PiSp = pi/SysPeak
    #Finding ADJUSTED AI
    adj_AI = 1 - AI
    #Input values for the final csv file
    extractedFeatures = [subject.iloc[patient, 2], SysPeak, AI, adj_AI, ev/av, PiSp, subject.iloc[patient, 15]]
    feature_vector.append(extractedFeatures)
    # #temp = pd.concat((subject.iloc[patient, :], extractedFeatures), axis=0)
    #feature_vector = pd.concat((feature_vector, temp), axis=0)

In [141]:
feat = pd.DataFrame(feature_vector)
feat.rename(columns = {0:'Age', 1:'Systolic Peak', 2:'Augmented Index', 3:'Adjusted AI', 4:'E/A', 5:'PI/SP', 6:'Label'}, inplace = True)

In [142]:
feat

,Age,Systolic Peak,Augmented Index,Adjusted AI,E/A,PI/SP,Label
0,66,0.821020,0.407008,0.592992,-0.095378,0.023153,1
1,52,0.731969,0.467247,0.532753,-0.085155,0.025957,1
2,63,0.701505,0.434116,0.565884,-0.116054,0.027084,1
3,59,0.655502,0.433824,0.566176,-0.085117,0.028999,1
4,55,0.905078,0.411376,0.588624,-0.084539,0.021003,1
...,...,...,...,...,...,...,...
92,24,0.672239,0.420533,0.579467,-0.082428,0.028277,0
93,24,0.770149,0.414720,0.585280,-0.095836,0.024670,0
94,25,0.681471,0.434331,0.565669,-0.097058,0.027881,0
95,25,0.792374,0.386894,0.613106,-0.092279,0.023978,0


In [143]:
# saving the DataFrame as a CSV file

feat.to_csv('FinalFeatures.csv', index = True)